## Applying Energy Path algtm to road photos

In [1]:
import cv2
import numpy as np
import matplotlib as plt

In [2]:
RED = (0, 0, 255)

In [3]:
img = cv2.imread("img.jpg")


In [4]:

cv2.imshow("original",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("gray",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
blurred = cv2.GaussianBlur(gray,(3,3),3)

cv2.imshow("blurred",blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Classic binary thresholding

In [7]:
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Adaptive binary thresholding

In [8]:
threshAdap = cv2.adaptiveThreshold(blurred, 255,
	cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 5)

cv2.imshow("adaptive thresh",threshAdap)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Morphological ops

In [9]:
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)
cv2.imshow("opened img",opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
def compute_energy(img):
    """
    calculeaza energia la fiecare pixel pe baza gradientului
    :param img: imaginea initiala
    :return:E - energia
    """
    # urmati urmatorii pasi:
    # 1. transformati imagine in grayscale
    # 2. folositi filtru sobel pentru a calcula gradientul in directia X si Y
    # 3. calculati magnitudinea pentru fiecare pixel al imaginii
    E = np.zeros((img.shape[0],img.shape[1]))
    # img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    grad_x = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=1,dy=0)
    grad_y = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=0,dy=1)

    abs_x = np.abs(grad_x)
    abs_y = np.abs(grad_y)

    E = abs_x + abs_y

    return E

In [12]:
gradient = compute_energy(opening)
cv2.imshow("gradient img",gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
def select_dynamic_programming_path(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i][j+1],M[i][j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    path = [0 for i in range(line+1)]

    path[line] = (line,col)
    
    for line in range(M.shape[0]-2,-1,-1):
        if col == 0:
            if M[line,0] < M[line,1]:
                new_col = 1
            else:
                new_col = 0
        elif col == E.shape[1] - 1:
            if M[line,col] < M[line,col-1]:
                new_col = col - 1
            else:
                new_col = col
        else:
            neigh = np.array([M[line,col-1],M[line,col],M[line,col+1]])
            new_col = col+np.argmax(neigh) - 1

        path[line]= (line,new_col)

        col = new_col

    return path

In [14]:
def create_path(img, pathL,pathR, color):
    
    new_image = img.copy()

    for row, col in pathL:
        new_image[row, col] = color

    for row, col in pathR:
        new_image[row, col] = color

    # E = compute_energy(img)
    # new_image_E = img.copy()
    # new_image_E[:,:,0] = E.copy()
    # new_image_E[:,:,1] = E.copy()
    # new_image_E[:,:,2] = E.copy()

    # for row, col in path:
    #     new_image_E[row, col] = color
    # cv2.imshow(name, np.uint8(new_image))
    return new_image
    # cv2.imshow('path E', np.uint8(new_image_E))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [15]:
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray,(3,3),3)

    _, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)

    return opening


In [16]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
cv2.imshow('gradient img', preR)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
gradL = compute_energy(preL)
gradR = compute_energy(preR)
pathL = select_dynamic_programming_path(gradL)
pathR = select_dynamic_programming_path(gradR)
pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])
# print(newPathR)
# cv2.imshow('preprocess img', pre)
cv2.imshow('gradientL img', gradL)
cv2.imshow('gradientR img', gradR)
new_img = create_path(img,pathL,pathR,RED)
cv2.imshow("New Image",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
cap = cv2.VideoCapture('C:/Programs/facultate/licenta/my-autonomous-car/resources/1.mp4')
 
while cap.isOpened():
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    pre = preprocess(frame)
    preL = pre[:,:int(pre.shape[1]/2)]
    preR = pre[:,int(pre.shape[1]/2):]
    gradL = compute_energy(preL)
    gradR = compute_energy(preR)
    pathL = select_dynamic_programming_path(gradL)
    pathR = select_dynamic_programming_path(gradR)
    pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])
    new_img = create_path(frame,pathL,pathR,RED)
    cv2.imshow("pre",pre)
    cv2.imshow('newImg', new_img)
    
    if cv2.waitKey(1) == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()